In [1]:
import time
import sys
import os
import torch
import json
import pandas as pd
import numpy as np

from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch import DeepAREstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.torch.distributions import StudentTOutput
from gluonts.torch.distributions import NormalOutput
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error

sys.path.append("../..")
from timefusion.utils import metrics

In [2]:
# Import dataset
train_data = pd.read_csv("../../datasets/electricity/train.csv", index_col="date")
val_data = pd.read_csv("../../datasets/electricity/val.csv", index_col="date")
test_data = pd.read_csv("../../datasets/electricity/test.csv", index_col="date")

# Normalize the signal power of each column
stds = train_data.std()
train_data /= stds
val_data /= stds
test_data /= stds

In [16]:
prediction_length = 24
freq = "H"

# Get training, validation and test dataset
train_flat = train_data.stack().reset_index()
train_flat.columns = ["date", "series", "value"]
train_dataset = PandasDataset.from_long_dataframe(train_flat, target="value",item_id="series",timestamp="date",freq=freq)

val_flat = val_data.stack().reset_index()
val_flat.columns = ["date", "series", "value"]
val_dataset = PandasDataset.from_long_dataframe(val_flat, target="value",item_id="series",timestamp="date",freq=freq)
val_dataset_14 = [PandasDataset.from_long_dataframe(val_flat.iloc[:-prediction_length*i*train_data.shape[1]] if i != 0 else val_flat, target="value",item_id="series",timestamp="date",freq=freq) for i in range(14)]

test_flat = test_data.stack().reset_index()
test_flat.columns = ["date", "series", "value"]
test_dataset_14 = [PandasDataset.from_long_dataframe(test_flat.iloc[:-prediction_length*i*train_data.shape[1]] if i != 0 else test_flat, target="value",item_id="series",timestamp="date",freq=freq) for i in range(14)]


In [17]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_dataset_14[0],
    predictor=predictor,
    num_samples=128
)

realisations = list(ts_it)

In [22]:
for i in test_data.iloc[-1]:
    print(i)

0.1992041784103127
3.365655433294702
0.1232222777938033
3.6183956979053655
3.9090674571131325
3.147622995261195
0.91387349625827
4.505770627027539
2.214092575860669
2.206989920609666
3.181874991866757
1.1596574548346101
1.8874218243148146
4.458261125587404
3.709190428197966
2.8841610203866885
3.686176588141712
3.9815788302198936
3.4742623574282394
1.7509587838282872
2.9270003711834445
3.33993347812057
3.189801243191601
3.4535335574643278
4.03509953677774
3.055911991280361
3.7925796765502695
5.31118126134664
4.27375261161761
0.47636248766032774
3.6040493635626016
3.3706397968436876
0.829403418305052
3.4372802348703275
3.05330735745446
3.7621456520751453
0.8731824159576943
3.909866968910951
3.2417726248234344
2.664106766750201
3.4951409307314396
3.0031907027338853
4.816273675936218
3.459838812751668
2.666715350174233
3.8875231597210558
1.599808839899204
3.84341602719762
1.9488920078639291
3.0430201967613524
5.415270192148953
4.328963846463478
3.2698997701308263
3.783065434406424
3.459944

In [23]:
len(realisations)

319

In [8]:
# Train the model and make predictions
model = DeepAREstimator(
    prediction_length = prediction_length, 
    freq=freq,
    context_length = 1*prediction_length,
    num_layers = 4,
    hidden_size = 30,
    lr = 1e-4,
    dropout_rate = 0.01,
    distr_output = StudentTOutput(),
    trainer_kwargs={"max_epochs": 1}
)

In [9]:
# Train model
predictor = model.train(training_data=train_dataset,validation_data=val_dataset,num_workers=3)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/rds/general/user/ejh19/home/Final-Year-Project/deepar/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/rds/general/user/ejh19/home/Final-Year-Project/deepar/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard sup

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 50: 'val_loss' reached 2.61888 (best 2.61888), saving model to '/rds/general/user/ejh19/home/Final-Year-Project/deepar/training_scripts/lightning_logs/version_696/checkpoints/epoch=0-step=50.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=1` reached.


In [57]:
# Validation test
samples = []
realisations = []

for dataset in test_dataset_14:
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=128
    )

    samples.append(list(forecast_it))
    realisations.append(list(ts_it))


In [73]:
# Convert to tensors
sample_tensor = torch.tensor(np.array([[l.samples for l in s] for s in samples])).permute(0,2,1,4)
realisations_tensor = torch.tensor([np.array(r).squeeze()[:,-prediction_length:] for r in realisations])

In [75]:
realisations_tensor.shape

torch.Size([14, 319, 24])

In [67]:
[np.array(r) for r in realisations]

[array([[[0.39840836],
         [0.59761254],
         [0.59761254],
         ...,
         [2.19124596],
         [2.19124596],
         [2.19124596]],
 
        [[3.79993355],
         [3.58279449],
         [3.14851637],
         ...,
         [5.31990698],
         [5.21133745],
         [5.86275463]],
 
        [[2.72732142],
         [2.89777901],
         [2.27276785],
         ...,
         [2.32958705],
         [2.38640624],
         [3.0682366 ]],
 
        ...,
 
        [[0.92918129],
         [0.62070815],
         [0.55299454],
         ...,
         [0.5341852 ],
         [1.39565287],
         [1.9260762 ]],
 
        [[0.80072958],
         [0.49471828],
         [0.50491865],
         ...,
         [0.82623052],
         [1.43825313],
         [2.91220758]],
 
        [[0.22289979],
         [0.15734103],
         [0.14422928],
         ...,
         [0.78670515],
         [1.5734103 ],
         [2.3994507 ]]]),
 array([[[0.39840836],
         [0.59761254],
         

In [37]:
k = [[l.samples for l in s] for s in samples]

In [40]:
np.array(k).shape

(14, 319, 128, 24)

In [35]:
samples[0]

[gluonts.model.forecast.SampleForecast(info=None, item_id='0', samples=array([[ 3.7165484,  2.7741966,  2.1837263, ...,  2.313342 ,  2.1021168,
          2.4074147],
        [ 2.2928956,  2.397815 ,  1.893832 , ...,  3.1192517,  2.5491495,
          3.2753413],
        [ 3.0082388,  1.4581267,  2.383847 , ...,  2.235709 ,  3.002548 ,
          1.416785 ],
        ...,
        [ 2.9890869,  2.3814719,  1.7632586, ...,  2.8404264,  2.7707798,
          2.9877691],
        [ 2.2510393,  1.9381064,  2.5961072, ...,  2.5898635,  1.7121726,
          2.6901186],
        [ 1.45507  ,  2.8097043,  1.3364005, ...,  2.6349952, -0.6781137,
          2.2773604]], dtype=float32), start_date=Period('2014-08-15 12:00', 'H')),
 gluonts.model.forecast.SampleForecast(info=None, item_id='1', samples=array([[5.019357 , 5.0283856, 4.851242 , ..., 5.644326 , 5.8420043,
         6.141685 ],
        [6.0254436, 5.8926425, 4.606315 , ..., 5.4682565, 5.2503357,
         6.535492 ],
        [6.084257 , 5.7994275

In [17]:
val_dataset_14[0]

PandasDataset<size=0, freq=H, num_feat_dynamic_real=0, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>

In [18]:
a = [1,2,3]
a[:-1]

[1, 2]